# ПРАКТИЧЕСКОЕ ЗАНЯТИЕ 5

В лекции 2 был рассмотрен алгоритм вычисления значения многочлена в точке по схеме Горнера
$$
    P_n(x)=a_0 \cdot x^n + a_1 \cdot x^{n-1} + ... + a_{n-1} \cdot x + a_n
$$

Будем считать, что задан массив коэффициентов $A=[a_0,a_1,...a_n]$, и некторое значение аргумента $x$. Тогда $F(A)=P_n(x).$

Соответствующий программный код на языке Julia выглядит так:

```julia
function eval_poly(x,A)
    Q = first(A) # - это есть a_0
    for a in @view A[2:end]
        Q=Q*x+a
    end
    return Q
end
```

Функция eval_poly(x,A) является обобщенной: реальный исполняемый код будет зависеть от фактических типов её аргументов:

```julia
eval_poly(5,[1,2,3,4]) # 5^3+2*5^2+3*5+4 - первый аргумент типа Int64
eval_poly(0.5,[1,2,3,4]) # 0.5^3+2*0.5^2+3*0.5+4 - первый аргумент типа Float64
eval_poly(1//2,[1,2,3,4]) # (1//2)^3+2*(1//2)^2+3*1//2+4  - первый аргумент типа Rational{Int64}
eval_poly(1+2im,[1,2,3,4]) # (1+2im)^3+2*(1+2im)^2+3*(1+2im)+4 - первый аргумент типа Complex{Int64}
eval_poly(0.5,1:4) # - второй аргумент типа UnitRange{Int64}
```

Многочлены можно вычислять и на множестве всех квадратных матриц, например:

```julia
A=[1 2 3
   4 5 6
   7 8 9]
eval_poly(A,1:4)
```

## Пользовательский тип, для оперирования с многочленами

К приведенным выше примерам можно было бы добавить еще пример вычисления многочлена от многочлена, (в результате должен получаться тоже многочлен, являющийся композицией первых двух).
Но для этого нужно, чтобы был определен специальный тип, для предсавления многочленов как объектов, и для этого типа должны быть определены операции `+` и `*`.

Существует пакет [`Polynomials.jl`](https://github.com/JuliaMath/Polynomials.jl), который содержит определение соответствующего типа `Polynomial` (этот пакет требуется установить). И эсли этот пакет импортировать, то задача вычисления многочлена от многочлена сразу будет решена. Однако в учебных целях реализуем свой собственный тип, назовем его `Polynom`.

```julia
struct Polynom{T} # это параметрический тип, позволяющий определять типы коэффициентов многочлена
    coeff::Vector{T} # вектор коэффициентов многочлена, заданных по убыванию степеней
end
```

Напомним, что в Julia структуры по умолчанию являются неизменяемыми. Если бы мы хотели иметь возможность изменять значения коэффициентов многочлена, то следовало бы определить тип так:

```julia
mutable struct Plynom{T} # это параметрический тип, позволяющий определять типы коэффициентов многочлена
   ....
end
```

Но если требуется максимальная эффективность исполняемого кода, то предпочтение следует отдавать не мутабельным типам.

Теперь для создания объектов определенного нами пользовательского типа `Polynom`, следует использовать конструктор этого типа (в данном случае мы используем конструктор по умолчанию):

```julia
p=Polynom([1,2,3,4])

```

Но чтобы теперь с многочленами можно было выполнять обычные операции сложения и умножения, эти операции нужно переопределить для для нашего типа:

```julia
module Polynoms
    import Base: +, * # это импортирование требуется, поскольку мы собираемся переопределить эти базовые опреации

    struct Polynom{T}
        coeff::Vector{T}
    end

    function +(p::Polynom, q::Polynom)
        np,nq = length(p.coeff), length(q.coeff)
        r=Vector{promote_type(eltype(p),eltype(q))}(undef,max(np,nq))
        if np >= nq
            r .= (@view p.coeff[end-nq+1:end]) .+ q.coeff  
        else
            r .= (@view q.coeff[end-np+1:end]) .+ p.coeff
        end
        return Polynom(r)
    end

#= ПОЯСНЕНИЯ
1. Здесь используется не векторизованное сложение массивов с формированием результата "на месте". Для этого использована запись соответствующих операций с точкой:  .=, .+. Например, запись
r .= (@view p.coeff[end-nq+1:end]) .+ q.coeff

есть просто краткая запись следующего цикла:

for i in length(p.coeff)-nq+1:length(p.coeff)
    r[i] = p.coeff[i] + q.coeff[i-length(p.coeff)+nq]
end

2. Здесь использована встроенная функция promote_type, которая получает на вход список типов и возвращает тот тип из этого списка, к которому автоматически будут приведиться все остальные при выполнении встроенных операций, если аргументы этих операций будут иметь разные типы.

Например, при сложении значений типов Float64 и Int64, второе значение автоматически будет приведено к типу Float64. 
=#

    function +(p::Polynom, c::Number)
        coeff=p.coeff
        coeff[end]+=c
        return Polynom(coeff)
    end

    +(c::Number, p::Polynom)=p+c

    function *(p::Polynom, q::Polynom)
        np,nq = length(p.coeff), length(q.coeff)
        coeff=zeros(promote_type(eltype(p),eltype(q)), np+nq-1)
        for i in eachindex(p.coeff), j in eachindex(q)
            coeff[i+j-1] += p.coeff[i]*q.coeff[j]
        end
        return Polynom(coeff)
    end

    function *(p::Polynom, c::Number)
        np,nq = length(p.coeff), length(q.coeff)
        coeff=zeros(promote_type(eltype(p),typeof(c)), length(p))
        for i in eachindex(p.coeff)
            coeff[i] += coeff[i]*c
        end
        return Polynom(coeff)
    end
    
    *(c::Number, p::Polynom) = p*c
end
```

Teперь уже можно будет вычислять значение многочлена от многочлена, например:

```julia

p=Plynom([1,2,3,4])
q=Plynom([1,2,1])

eval_poly(q,p) # - новый многочлен, представляющий собой композицию двух данных многочленов
```

### Вызов объектов как функций

Если у нас имеется объект типа `Polynom`, например:

```julia
p=Polynom([1,2,1])
```

то было бы естественно рассматривать объетк `p` как функцию. Т.е. если, например, требуется вычислить значение этого многочлена в точке `х=0.5`, то хотелось бы иметь возможность написать просто:

```julia
p(0.5)
```

вместо того, чтобы вызывать функцию `eval_poly` (или аналогичную встоенную функцию `evalpoly`):

```julia
eval_poly(0.5, p.coeff)
```

Чтобы получить такую возможность необходимо определить возможность вызова объекта `Polynom` как функцию, т.е. возможность использования операции вызова `()`  для объектов этого типа. Делается это следующим образом.

```julia
function (p::Polynom, x) 
    return eval_poly(x,p.coeff)
end
```

Или в данном конкретном случае это можно было бы сделать короче:

```julia
(p::Polynom, x) = eval_poly(x,p.coeff)
```

### Вычисление значения производной многочлена в точке

В лекции 2 было построено соответствующее индуктивное расширение функции - значения производной многочлена в точке, вычисленное на последовательности его коэффициентов, и реализована соответствующий программный код:

```julia
function evaldiffpoly(x,A)
    Q′=0
    Q=0
    for a in A
        Q′=Q′x+Q
        Q=Q*x+a
    end
    return Q′,Q
end
```

Задача 1. Аналогично построить индуктивную функцию и реализовать соответствующий программный код, вычисляющий значение второй производной многочлена в точке.

Задача 2. Сделать то же самое для 3-ей производной многочлена, заданного своими коэффициентами

Задача 3. Сделать то же самое для k-ой производной многочлена, заданного своими коэффициентами (здесь предполагается, что  `k` - это параметр функции `evaldiffpoly`)

Задача 4. Определить функцицию с именем `diff`, для пользовательского типа `Polynom`, так чтобы ее можно было бы использовать так:

```julia
p=Polynom([....])
diff(p,x; ord=2) # ord - порядок производной
```

при этом по умолчанию именованный аргумент `ord` сделать равным 1.

### Деление многочлена на многочлен

Реализовать функцию деления многочлена на многочлен, получающую на вход два массива с коэффициентами этих многочленов, и возвращающую другие два массива с коэффициентами частного и остатка. 

Указание. Воспользоваться алгоритмом деления "уголком". Сначала ограничиться случаем многочленов с целочисленными вещественными коэффициентами, а затем подумать, в чем будет состоять особенность такого деления, если допустить значения коэффициентов с плавающей точкой.

Задача 5. Соответствующим образом переопределить операцию деления `/` для пользовательского типа `Polynom`.



### Однопроходный алгоритм вычисления дисперсии последовательности

В лекции 2 было построено требуемое индуктивное расширение и реализован соответствующий однопроходный алгоритм, вычисляющий дисперсию заданной последовательности:

```julia
function dispersion(series)
    S¹ = eltype(series)(0)
    S² = eltype(series)(0)
    for (n,a) in enumerate(series)
        S¹ += a
        S² += a^2
        M = S¹/n
        D = S²/n-M^2
    end
    return D, M
end
```

Задача 6. Пусть задан массив данных `a=[a[1],...,a[N]]`. Написать функцию `currenstd`, получающую на вход некоторую последовательность, и возвращающую массив оценок "текущего" значения стандартного отклонения `std` (std=sqrt(D)), получаемых для по первым `n` членам последовательности `a` (`n=1,2,...N`). Данный алгоритм должен быть онопроходным.

Для тестирования этой функции массив данных подготовить с использованием встроенной функции `randn` следующим образом.

Для справки, функция `randn` формирует массив заданного типа и заданного размера из независимых псевдослучайных чисел, распределенных по так называемому гауссовскому закону. В общем случае гауссовское распределение характеризуется двумя параметрами: математическим ожиданием (общепринятое обозначение - $m$) и средним квадратическим отклонением (общепринятое обозначение - $\sigma$). Функция `random` генериует числовые значения с нулевым математическим ожиданием и среднимквадратическим отклонением, равным 1. Получить массив данных заданной длины $n$, в котором среднее квадратическое имеет заданное значение $\sigma$ и заданное математическое ожидание $m$, можно так:

```julia
a=randn(Float64,n) .* σ + m
```

При этом имеет место следующий статистический факт. 
Пусть

```julia
D, M = dispersion(a)
```

тогда, при достаточно больших $n$ $sqrt(D_n)$ приблизительно равно σ, а $M_n$ будет приблизително равно $m$.
И в пределе при $n \rightarrow \infty \   D_n \rightarrow \sigma, M_n \rightarrow m$.

Убедиться в этом экспериментально, построив график зависимостей  $D_n, M_n$ для $n=1,2,...,N$, взяв значение $N$ достаточно большим (подобрать его экспериментально, так чтобы четко проявились горизонтальные асимптоты графиков).

Для построения графиков воспользоваться функцией `plot` из пакета `Plots.jl` (который требует предварительной установки). Напомним, что этот пакет реализует универсальный интерфейс из различных графичеких бэкендов, в частности можно воспользоваться бэкендом `pyplot`, для подключения которого потребуется вызвать функцию `pyplot()`:

```julia
using Plots
pyplot()
```
